In [ ]:
# Clean install with all required deps
!pip install -U torch transformers sentence-transformers faiss-cpu flask pyngrok accelerate bitsandbytes



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.6/486.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 5.1.0
    Uninstalling sentence-transformers-5.1.0:
      Successfully uninstalled sentence-transformers-5.1.0


In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import faiss
from flask import Flask, request, jsonify
from pyngrok import ngrok




In [ ]:
# Cell 3: Knowledge Base

knowledge_base = [
    {
        "id": "Doc1",
        "title": "Shoplite User Registration and Account Management",
        "content": """
        Shoplite allows users to create buyer or seller accounts.
        Buyers register with email, password, and profile info, requiring email verification within 24 hours.
        Sellers must complete business verification (tax ID, licenses, bank info), which takes 2–3 business days.
        Both buyers and sellers can manage settings like addresses, payment methods, and notification preferences.
        """
    },
    {
        "id": "Doc2",
        "title": "Shoplite Product Search and Filtering Features",
        "content": """
        Shoplite provides a universal search bar supporting keywords, auto-suggestions, and trending terms.
        Filters allow refinement by price, rating, category, delivery time, and location.
        Advanced filters exist for categories like fashion (size, color) and electronics (processor, memory).
        Results can be sorted by relevance, price, newest, or top-rated, with mobile voice search supported.
        """
    },
    {
        "id": "Doc3",
        "title": "Shoplite Shopping Cart and Checkout Process",
        "content": """
        Users can add items from multiple sellers to their shopping cart.
        Cart contents sync across sessions for logged-in users.
        Checkout involves confirming addresses, selecting delivery, applying promo codes, and reviewing summaries.
        Order confirmation numbers and emails are generated after purchase.
        """
    },
    {
        "id": "Doc4",
        "title": "Shoplite Payment Methods and Security",
        "content": """
        Shoplite supports credit/debit cards, PayPal, Apple Pay, Google Pay, and bank transfers.
        All transactions use SSL encryption and PCI-DSS compliant gateways with tokenization.
        Two-factor authentication may be required for high-risk transactions.
        Payment activity alerts notify users of unusual activity for added security.
        """
    },
    {
        "id": "Doc5",
        "title": "Shoplite Order Tracking and Delivery",
        "content": """
        Customers can track orders using tracking numbers in their account dashboard or confirmation email.
        Status updates include shipment progress, customs clearance for international orders, and estimated delivery times.
        Notifications are sent via email, SMS, and push alerts when order statuses change.
        """
    },
    {
        "id": "Doc6",
        "title": "Shoplite Return and Refund Policies",
        "content": """
        Customers can request returns within a 30-day return window by generating a return authorization number (RMA).
        Returned items must be inspected before refunds are issued.
        Refunds are processed within 5–7 business days to the original payment method.
        Replacement items may be offered instead of refunds upon request.
        """
    },
    {
        "id": "Doc7",
        "title": "Shoplite Product Reviews and Ratings",
        "content": """
        Verified buyers can leave star ratings (1–5) and written feedback.
        Reviews may cover product quality, delivery experience, and seller communication.
        Sellers can publicly respond to reviews to resolve issues or thank customers.
        Fake or abusive reviews are moderated and may be removed.
        """
    },
    {
        "id": "Doc8",
        "title": "Shoplite Seller Account Setup and Management",
        "content": """
        Sellers register by submitting business verification details such as tax ID, licenses, and bank account information.
        Verification takes 2–3 business days before sellers can access the Seller Dashboard.
        From the dashboard, sellers can list products, manage inventory, and track sales.
        """
    },
    {
        "id": "Doc9",
        "title": "Shoplite Inventory Management for Sellers",
        "content": """
        The Seller Dashboard allows manual product listing or bulk uploads via CSV.
        Inventory is tracked in real time, with automatic low-stock alerts.
        Sellers can integrate via API with third-party systems for inventory syncing.
        Logs and analytics provide insights into product availability and demand planning.
        """
    },
    {
        "id": "Doc10",
        "title": "Shoplite Commission and Fee Structure",
        "content": """
        Shoplite charges a commission between 5% and 15% depending on product category.
        An additional 1%–3% payment processing fee applies to each transaction.
        Sellers may pay optional fees for promoted listings and marketing campaigns.
        Payouts are issued on a 7–14 day schedule, with full breakdowns shown in the Seller Dashboard.
        """
    },
    {
        "id": "Doc11",
        "title": "Shoplite Customer Support Procedures",
        "content": """
        Customers can contact support via live chat, email, or the self-service help center.
        Tickets can be submitted for complex issues and are resolved within 24–48 hours.
        Escalation paths exist for unresolved or critical cases.
        Support interactions are logged for accountability and follow-up.
        """
    },
    {
        "id": "Doc12",
        "title": "Shoplite Mobile App Features",
        "content": """
        The Shoplite mobile app (iOS/Android) provides push notifications, offline cart functionality, and biometric login.
        Users can access exclusive in-app discounts and reorder frequently purchased items quickly.
        Sellers can manage orders, respond to customer messages, and track sales from the app.
        Accessibility features include screen reader support, text resizing, and high-contrast mode.
        """
    },
    {
        "id": "Doc13",
        "title": "Shoplite API Documentation for Developers",
        "content": """
        Shoplite provides a REST-based API with OAuth 2.0 authentication and SSL security.
        Developers can manage listings, process orders, update accounts, and fetch analytics.
        API usage is rate-limited to 1000 requests per hour, with higher limits for approved partners.
        A sandbox environment is available for testing integrations before production.
        """
    },
    {
        "id": "Doc14",
        "title": "Shoplite Security and Privacy Policies",
        "content": """
        Shoplite uses SSL encryption for all traffic and complies with PCI-DSS and GDPR standards.
        Customers may enable two-factor authentication and review login activity.
        Accounts showing suspicious behavior may be temporarily locked.
        Users can request account deletion or export under privacy rights.
        """
    },
    {
        "id": "Doc15",
        "title": "Shoplite Promotional Codes and Discounts",
        "content": """
        Promotional codes can be applied at checkout for percentage discounts, fixed-amount reductions, or free shipping.
        Codes may require minimum purchases, expire after certain dates, or be limited to one use per account.
        Some promotions are platform-wide, while others are seller-specific.
        Sellers can monitor usage data from their dashboard.
        """
    },
    {
        "id": "Doc16",
        "title": "Shoplite Fraud Prevention and Account Safety",
        "content": """
        Shoplite uses machine learning models to detect fraud based on abnormal patterns.
        Suspicious logins or transactions may trigger alerts or require two-factor authentication.
        Verified sellers undergo periodic identity checks.
        Users can report fraudulent listings, which are reviewed within 24 hours.
        """
    },
    {
        "id": "Doc17",
        "title": "Shoplite Notifications and Communication Channels",
        "content": """
        Customers receive notifications for order confirmations, shipments, and promotions via email, SMS, and push alerts.
        Sellers are notified of new orders, low inventory, and policy updates.
        Notification preferences can be managed in Account Settings, but essential alerts cannot be disabled.
        The Seller Dashboard also includes a message center for direct buyer-seller communication.
        """
    }
]


In [ ]:
PROMPTS = {
    "base_retrieval_prompt": {
        "role": "You are a helpful Shoplite customer service assistant.",
        "goal": "Provide accurate answers using only the provided Shoplite documentation.",
        "context_guidelines": [
            "Use only information from the provided document snippets",
            "Cite specific documents when possible"
        ],
        "response_format": "Answer: [Your response based on context]\n  Sources: [DO NOT invent titles. Use ONLY the list provided after the question.]"

    },
    "multi_doc_prompt": {
    "role": "You are a Shoplite assistant handling complex questions.",
    "goal": "Combine information from multiple documents when needed to answer questions.",
    "context_guidelines": [
        "ONLY use information from documents directly relevant to the query.",
        "Completely ignore and exclude documents that are not relevant.",
        "Do NOT mention documents that were ignored.",
        "If multiple documents provide relevant details, merge them logically into one clear answer.",
        "Cite ONLY the documents you actually used."
    ],
      "response_format": "Answer: [Combined response from documents]\nSources: [List ONLY the document titles actually used]"
},

    "refusal_prompt": {
        "role": "You are a careful Shoplite assistant.",
        "goal": "Refuse to answer if no relevant context is found in the knowledge base.",
        "context_guidelines": [
            "Do not make up answers",
            "Politely explain that no relevant information exists in the documentation"
        ],
        "response_format": "Answer: I'm sorry, I don’t have that information in the Shoplite documentation.\nSources: Only say None"
    },
    "clarification_prompt": {
        "role": "You are a Shoplite assistant that ensures user questions are clear.",
        "goal": "Ask the user for clarification if their question is ambiguous.",
        "context_guidelines": [
            "Identify when a query has multiple interpretations or missing details",
            "Prompt the user to provide more specifics before answering"
        ],
        "response_format": "Answer: Could you clarify your request?\nSources: ONLY say None"
    }
}


In [ ]:
# Login for the hugging face model
from huggingface_hub import login

login()

In [ ]:
#LLM Loading and setup

model_id = "meta-llama/Llama-3.1-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)
print(f"Model {model_id} loaded successfully with 4-bit quantization")




NameError: name 'BitsAndBytesConfig' is not defined

In [ ]:
# === RAG PIPELINE FUNCTIONS ===

def create_embeddings(knowledge_base):
    embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    docs = [doc["content"] for doc in knowledge_base]
    embeddings = embedder.encode(docs, convert_to_numpy=True, show_progress_bar=True)

    # Build FAISS index
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)

    # Map FAISS IDs → documents
    id_map = {i: knowledge_base[i] for i in range(len(knowledge_base))}
    return index, id_map, embedder


def retrieve_docs(query, index, id_map, embedder, top_k=2):
    query_emb = embedder.encode([query], convert_to_numpy=True)
    scores, indices = index.search(query_emb, top_k)
    results = [id_map[i] for i in indices[0]]
    return results

def filter_docs_by_keywords(query, docs):
    """
    Rank and filter documents by overlap with query terms.
    Uses a stricter overlap rule to avoid irrelevant docs.
    """
    # Tokenize query
    q_words = set(query.lower().split())
    scored = []

    for doc in docs:
        doc_text = (doc["title"] + " " + doc["content"]).lower()

        # Count how many query words appear in doc
        overlap = sum(1 for w in q_words if w in doc_text)

        # Basic lexical filter: require at least 1 overlap
        if overlap > 0:
            scored.append((overlap, doc))

    # Sort docs by overlap score (descending)
    scored.sort(key=lambda x: x[0], reverse=True)

    # Return only the docs (strip scores)
    filtered = [doc for _, doc in scored]

    # Fallback: if nothing passed the filter, return original docs
    return filtered if filtered else docs


def generate_response(
    query,
    index,
    id_map,
    embedder,
    prompt_key=None,
    top_k=4
):
    #  Retrieve documents
    retrieved_docs = retrieve_docs(query, index, id_map, embedder, top_k)
    retrieved_docs = filter_docs_by_keywords(query, retrieved_docs)

    # Auto-select prompt if not provided
    if prompt_key is None:
        if not retrieved_docs:  # no matches at all
            prompt_key = "refusal_prompt"
        elif len(query.split()) < 3:  # very short / vague
            prompt_key = "clarification_prompt"
        elif len(retrieved_docs) > 1:  # multiple relevant docs
            prompt_key = "multi_doc_prompt"
        else:
            prompt_key = "base_retrieval_prompt"

    #Hard guardrails
    if prompt_key == "refusal_prompt":
        return {
            "query": query,
            "answer": "I'm sorry, I don’t have that information in the Shoplite documentation.",
            "sources": []
        }

    if prompt_key == "clarification_prompt":
        return {
            "query": query,
            "answer": "Could you clarify your request?",
            "sources": []
        }

    # Multi-doc narrowing
    if prompt_key == "multi_doc_prompt":
      retrieved_docs = retrieve_docs(query, index, id_map, embedder, top_k=6)
      retrieved_docs = filter_docs_by_keywords(query, retrieved_docs)
      retrieved_docs = retrieved_docs[:2]  # cap at 2 best matches

    # Build context + sources
    context = "\n\n".join([doc["content"] for doc in retrieved_docs])
    sources = [doc["title"] for doc in retrieved_docs]

    # Choose prompt config
    if len(retrieved_docs) > 1 and prompt_key == "base_retrieval_prompt":
        prompt_config = PROMPTS["multi_doc_prompt"]
    else:
        prompt_config = PROMPTS[prompt_key]

    # Build structured prompt
    sources_titles = ", ".join([doc["title"] for doc in retrieved_docs])

    prompt = f"""
{prompt_config['role']}

{prompt_config['goal']}

Guidelines:
{chr(10).join(prompt_config['context_guidelines'])}

Context:
{context}

Question: {query}

Use the following document titles when filling Sources:
{sources_titles}

Answer in complete sentences, following this format:
{prompt_config['response_format']}
"""


    # Generate deterministically
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=250,
        eos_token_id=tokenizer.eos_token_id,
        temperature=0.0,
        do_sample=False
    )
    raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Post-process
    # Step 5: Post-process answer
    answer = raw_output

    # Keep only the text after "Answer:" if it exists
    if "Answer:" in answer:
        answer = answer.split("Answer:")[-1].strip()

    # Remove the query text if it got echoed
    if query in answer:
        answer = answer.replace(query, "").strip()


    sources = [doc["title"] for doc in retrieved_docs]

    return {
        "query": query,
        "answer": answer.strip(),
        "sources": [doc["title"] for doc in retrieved_docs]}





index, id_map, embedder = create_embeddings(knowledge_base)


print("\n=== Base Prompt Test ===")
q1 = "How do I create a seller account on Shoplite?"
r1 = generate_response(q1, index, id_map, embedder, prompt_key="base_retrieval_prompt")
print("Q:", r1["query"])
print("Answer:", r1["answer"])
print("Sources:", r1["sources"])

print("\n=== Multi-Doc Prompt Test ===")
q2 = "What are Shoplite's return policies and how do I track my order?"
r2 = generate_response(q2, index, id_map, embedder, prompt_key="multi_doc_prompt", top_k=3)
print("Q:", r2["query"])
print("Answer:", r2["answer"])
print("Sources:", r2["sources"])

print("\n=== Refusal Prompt Test ===")
q3 = "Does Shoplite offer scholarships?"
r3 = generate_response(q3, index, id_map, embedder, prompt_key="refusal_prompt")
print("Q:", r3["query"])
print("Answer:", r3["answer"])
print("Sources:", r3["sources"])

print("\n=== Clarification Prompt Test ===")
q4 = "How do I update my account?"
r4 = generate_response(q4, index, id_map, embedder, prompt_key="clarification_prompt")
print("Q:", r4["query"])
print("Answer:", r4["answer"])
print("Sources:", r4["sources"])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


=== Base Prompt Test ===


UnboundLocalError: cannot access local variable 'filtered' where it is not associated with a value

In [ ]:
# Flask API setup with prompt_key support
from flask import Flask, request, jsonify
import threading
import requests

app = Flask(__name__)

@app.route('/chat', methods=['POST'])
def chat():
    data = request.get_json()
    query = data.get("query")
    prompt_key = data.get("prompt_key")  # optional manual override

    if not query:
        return jsonify({"error": "Missing 'query'"}), 400

    try:
        result = generate_response(query, index, id_map, embedder, prompt_key=prompt_key)
        return jsonify(result)
    except Exception as e:
        return jsonify({"error": str(e)}), 500



@app.route('/ping', methods=['POST'])
def ping():
    data = request.get_json()
    prompt = data.get("prompt")

    if not prompt:
        return jsonify({"error": "Missing 'prompt'"}), 400

    try:
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=100, eos_token_id=tokenizer.eos_token_id)
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return jsonify({"prompt": prompt, "answer": answer})
    except Exception as e:
        return jsonify({"error": str(e)}), 500


@app.route('/health', methods=['GET'])
def health():
    try:
        return jsonify({
            "device": str(model.device),
            "torch_cuda": torch.cuda.is_available(),
            "model_loaded": True,
            "model_type": model.name_or_path,
            "index_size": len(id_map)
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500


In [ ]:
#ngrok token input and tunnel creation

from threading import Thread

ngrok_token = input("Enter your ngrok token: ")

ngrok.set_auth_token(ngrok_token)

# Kill old tunnels (if any) to avoid conflicts
ngrok.kill()

public_url = ngrok.connect(5000)  #
print("🚀 Flask API is live at:", public_url)

def run_flask():
    app.run(port=5000, use_reloader=False)

Thread(target=run_flask, daemon=True).start()



Enter your ngrok token: 33QqiYpX3Rq97FY5M5SoH1CVnB1_jpEPdmBnNQhkw7HTB6sn
🚀 Flask API is live at: NgrokTunnel: "https://jack-unvanquished-marlyn.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [ ]:
# Testing API Ednpoints and validation
import requests, json

public_url = public_url.public_url


def show(title, resp):
    print(f"\n=== {title} ===")
    try:
        print(resp.json())
    except Exception:
        print(resp.text)

# Health check
show("Health", requests.get(f"{public_url}/health"))

# Ping test
show("Ping", requests.post(f"{public_url}/ping", json={"prompt": "Say hello"}))

# Base chat test
show("Chat (base)", requests.post(f"{public_url}/chat", json={
    "query": "How do I create a seller account on Shoplite?",
    "prompt_key": "base_retrieval_prompt"
}))

# Multi-doc chat test
show("Chat (multi-doc)", requests.post(f"{public_url}/chat", json={
    "query": "What are Shoplite's return policies and how do I track my order?",
    "prompt_key": "multi_doc_prompt"
}))

# Refusal chat test
show("Chat (refusal)", requests.post(f"{public_url}/chat", json={
    "query": "Does Shoplite offer scholarships?",
    "prompt_key": "refusal_prompt"
}))

# Clarification chat test
show("Chat (clarification)", requests.post(f"{public_url}/chat", json={
    "query": "How do I update my account?",
    "prompt_key": "clarification_prompt"
}))


INFO:werkzeug:127.0.0.1 - - [02/Oct/2025 13:16:24] "GET /health HTTP/1.1" 200 -



=== Health ===
{'device': 'cuda:0', 'index_size': 17, 'model_loaded': True, 'model_type': 'meta-llama/Llama-3.1-8B-Instruct', 'torch_cuda': True}


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [02/Oct/2025 13:16:33] "POST /ping HTTP/1.1" 200 -



=== Ping ===
{'answer': 'Say hello to the newest member of the family – our baby girl, Aurora! She was born on October 30th, 2023, and weighed in at 7 pounds 1 ounce. We are absolutely over the moon with joy and can’t wait to watch her grow and explore the world.\nAurora is already bringing so much love and light into our lives, and we feel incredibly blessed to have her as our little one. We can’t wait to see the amazing things she will do and', 'prompt': 'Say hello'}


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [02/Oct/2025 13:16:44] "POST /chat HTTP/1.1" 200 -


...


=== Chat (multi-doc) ===
{'answer': "To initiate a return, customers can request a return authorization number (RMA) within a 30-day return window. Once the returned items are inspected, refunds are processed within 5–7 business days to the original payment method. If desired, replacement items may be offered instead of refunds. However, I couldn't find any information on how to track your order. Sources: Return Policy, Refund Process.  Note: The information on tracking orders was not found in the provided documents. \n\nNote: Please let me know if you need any further assistance! \n\n(Also, I'll be happy to help you with any other questions you may have!)", 'query': "What are Shoplite's return policies and how do I track my order?", 'sources': ['Shoplite Return and Refund Policies', 'Shoplite Fraud Prevention and Account Safety']}


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [02/Oct/2025 13:16:58] "POST /chat HTTP/1.1" 200 -



=== Chat (refusal) ===
{'answer': "I'm sorry, I don’t have that information in the Shoplite documentation.\nSources: None\nReason: No relevant context was found in the knowledge base.", 'query': 'Does Shoplite offer scholarships?', 'sources': ['Shoplite Payment Methods and Security', 'Shoplite Commission and Fee Structure']}


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [02/Oct/2025 13:17:19] "POST /chat HTTP/1.1" 200 -



=== Chat (clarification) ===
{'answer': "Could you clarify your request?\nSources: None\n \nExplanation: Your question is ambiguous because it could refer to updating your account information, updating your order status, or updating your shipping address. To provide a relevant answer, I need more specifics about what you are trying to update. Please let me know if you are looking to update your account information, order status, or shipping address. I'll be happy to assist you once I have more clarity on your request. \n\nSources: None\nExplanation: Your question is ambiguous because it could refer to updating your account information, updating your order status, or updating your shipping address. To provide a relevant answer, I need more specifics about what you are trying to update. Please let me know if you are looking to update your account information, order status, or shipping address. I'll be happy to assist you once I have more clarity on your request. \n\nSources: None\nExpla